In [12]:
import dask.dataframe as dd
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

/Users/saitejasriyerramsetti/.pyenv/versions/3.11.7/lib/python3.11/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [16]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

# set neat seaborn whitegrid styles for matplotlib charts

sns.set_style('whitegrid')

In [6]:
df = pd.read_csv('/Users/saitejasriyerramsetti/Documents/BDA 696/chicago crime/data/Crimes_-_2001_to_Present_20240918.csv')

In [7]:
df.shape

(8156508, 22)

In [10]:
import requests
import pandas as pd
from datetime import datetime

# Base URL for Chicago Crimes API
base_url = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"

# The last date from which we want to fetch new data (previously saved)
# For example, if the last data fetch included crimes up to 2024-09-30
last_fetched_date = "2024-09-30T00:00:00.000"  # Replace this with the actual last date you fetched

# API parameters to fetch data after the last fetched date
params = {
    "$limit": 1000,  # Maximum rows per request
    "$where": f"date > '{last_fetched_date}'"  # Fetch crimes after the last fetched date
}

# Fetch the new data
response = requests.get(base_url, params=params)
new_data = response.json()

# Convert the new data to a Pandas DataFrame
df_new = pd.DataFrame(new_data)

# Append the new data to the existing dataset (assume you have an existing DataFrame df_existing)
df_combined = pd.concat([df, df_new], ignore_index=True)

# Save the updated dataset (optional)
df_combined.to_csv("chicago_crimes_updated.csv", index=False)

print(f"Total rows after update: {len(df_combined)}")


Total rows after update: 8157508


In [20]:
print('Chicago Crime data stats:')
print('---------------------------------------')
print('{:,} total records'.format(len(df)))  # No partitions for pandas DataFrame
print('DataFrame size: {:,}'.format(df.size))


Chicago Crime data stats:
---------------------------------------
8,156,508 total records
DataFrame size: 179,443,176


In [21]:
df.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11037294,JA371270,03/18/2015 12:00:00 PM,0000X W WACKER DR,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,BANK,False,False,...,42.0,32.0,11,NaN,NaN,2015,08/01/2017 03:52:26 PM,NaN,NaN,NaN
1,11646293,JC213749,12/20/2018 03:00:00 PM,023XX N LOCKWOOD AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,False,False,...,36.0,19.0,11,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
2,11645836,JC212333,05/01/2016 12:25:00 AM,055XX S ROCKWELL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,15.0,63.0,11,NaN,NaN,2016,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,11645959,JC211511,12/20/2018 04:00:00 PM,045XX N ALBANY AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,...,33.0,14.0,08A,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
4,11645601,JC212935,06/01/2014 12:01:00 AM,087XX S SANGAMON ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,21.0,71.0,11,NaN,NaN,2014,04/06/2019 04:04:43 PM,NaN,NaN,NaN


In [22]:
import pandas as pd

# Load your data (assuming df is your DataFrame)
# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Extract different components of the date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.day_name()
df['Hour'] = df['Date'].dt.hour

# Quick overview of data types and null values
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8156508 entries, 0 to 8156507
Data columns (total 25 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ID                    int64         
 1   Case Number           object        
 2   Date                  datetime64[ns]
 3   Block                 object        
 4   IUCR                  object        
 5   Primary Type          object        
 6   Description           object        
 7   Location Description  object        
 8   Arrest                bool          
 9   Domestic              bool          
 10  Beat                  int64         
 11  District              float64       
 12  Ward                  float64       
 13  Community Area        float64       
 14  FBI Code              object        
 15  X Coordinate          float64       
 16  Y Coordinate          float64       
 17  Year                  int32         
 18  Updated On            object        
 19  

In [23]:
df.describe()

,ID,Date,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Month,Hour
count,8.156508e+06,8156508,8.156508e+06,8.156461e+06,7.541679e+06,7.543055e+06,8.066667e+06,8.066667e+06,8.156508e+06,8.066667e+06,8.066667e+06,8.156508e+06,8.156508e+06
mean,7.311930e+06,2011-01-23 02:22:04.019506176,1.184680e+03,1.129756e+01,2.277397e+01,3.742005e+01,1.164628e+06,1.885864e+06,2.010559e+03,4.184241e+01,-8.767139e+01,6.535482e+00,1.309434e+01
min,6.340000e+02,2001-01-01 00:00:00,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03,3.661945e+01,-9.168657e+01,1.000000e+00,0.000000e+00
25%,3.939192e+06,2005-04-14 23:00:00,6.210000e+02,6.000000e+00,1.000000e+01,2.300000e+01,1.153028e+06,1.859136e+06,2.005000e+03,4.176885e+01,-8.771350e+01,4.000000e+00,9.000000e+00
50%,7.307124e+06,2010-01-01 11:06:30,1.034000e+03,1.000000e+01,2.300000e+01,3.200000e+01,1.166168e+06,1.890878e+06,2.010000e+03,4.185622e+01,-8.766571e+01,7.000000e+00,1.400000e+01
75%,1.059528e+07,2016-07-03 23:05:15,1.731000e+03,1.700000e+01,3.400000e+01,5.700000e+01,1.176390e+06,1.909379e+06,2.016000e+03,4.190702e+01,-8.762811e+01,9.000000e+00,1.900000e+01
max,1.360058e+07,2024-09-10 00:00:00,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,1.205119e+06,1.951622e+06,2.024000e+03,4.202291e+01,-8.752453e+01,1.200000e+01,2.300000e+01
std,3.665571e+06,NaN,7.035513e+02,6.959836e+00,1.385599e+01,2.154698e+01,1.697433e+04,3.246270e+04,6.734973e+00,8.931732e-02,6.150334e-02,3.338670e+00,6.756705e+00


In [24]:
df.isnull().sum()


ID                           0
Case Number                  0
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description     13553
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614829
Community Area          613453
FBI Code                     0
X Coordinate             89841
Y Coordinate             89841
Year                         0
Updated On                   0
Latitude                 89841
Longitude                89841
Location                 89841
Month                        0
DayOfWeek                    0
Hour                         0
dtype: int64

In [25]:
df['Location Description'].value_counts().head(10)

Location Description
STREET                            2131943
RESIDENCE                         1350410
APARTMENT                          948674
SIDEWALK                           748670
OTHER                              269994
PARKING LOT/GARAGE(NON.RESID.)     202957
ALLEY                              181610
SMALL RETAIL STORE                 159159
SCHOOL, PUBLIC, BUILDING           146372
RESIDENCE-GARAGE                   135466
Name: count, dtype: int64

In [26]:
df['Location Description'].isnull().sum()

np.int64(13553)

In [27]:
# View rows where 'Location Description' is missing
missing_location_desc = df[df['Location Description'].isnull()]
print(missing_location_desc.head())


          ID Case Number                Date                 Block  IUCR  \
2   11645836    JC212333 2016-05-01 00:25:00   055XX S ROCKWELL ST  1153   
9   11645833    JC213044 2012-05-05 12:25:00       057XX W OHIO ST  1153   
23  11646766    JC214025 2018-01-01 09:10:00   019XX N KENMORE AVE  1153   
57  11648818    JC216121 2016-05-08 09:45:00       0000X W 73RD ST  1153   
60  11648806    JC216119 2013-12-01 09:20:00  001XX E MARQUETTE RD  1153   

          Primary Type                          Description  \
2   DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   
9   DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   
23  DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   
57  DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   
60  DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   

   Location Description  Arrest  Domestic  ...  X Coordinate  Y Coordinate  \
2                   NaN   False     False  ...           NaN           NaN   
9       

In [30]:
df.isna().sum().sum()

np.int64(1691087)

In [34]:
# Group by 'Block' to see the most common location description for each block
block_location_mapping = df.groupby('Block')['Location Description'].agg(lambda x: x.value_counts())

# # Apply this mapping to fill missing values in 'Location Description'
# df['Location Description'] = df.apply(
#     lambda row: block_location_mapping.get(row['Block']) if pd.isnull(row['Location Description']) else row['Location Description'], 
#     axis=1
# )


In [35]:
block_location_mapping

Block
0000X E 100 PL                                           [18, 17, 6, 2, 1]
0000X E 100 ST                             [21, 9, 4, 3, 2, 1, 1, 1, 1, 1]
0000X E 100TH PL         [187, 179, 94, 28, 27, 20, 14, 8, 6, 5, 5, 4, ...
0000X E 100TH ST         [170, 143, 61, 38, 24, 18, 8, 6, 5, 5, 3, 3, 2...
0000X E 101 PL                                                   [6, 3, 1]
                                               ...                        
139XX S ATLANTIC                                                         1
145XX S MINERVA                                                          1
173XX  LORENZ                                                            1
175XX S SANDALWOOD DR                                                    1
XX  UNKNOWN                                                         [1, 1]
Name: Location Description, Length: 64181, dtype: object

In [36]:
# Check length of lists for each block
df['list_length'] = df['Block'].apply(lambda x: len(x))
print(df[['Block', 'list_length']].head())


                  Block  list_length
0     0000X W WACKER DR           17
1  023XX N LOCKWOOD AVE           20
2   055XX S ROCKWELL ST           19
3    045XX N ALBANY AVE           18
4   087XX S SANGAMON ST           19


In [37]:
# Sum the counts in each list for each block
df['total_counts'] = df['Block'].apply(lambda x: sum(x) if isinstance(x, list) else 0)
print(df[['Block', 'total_counts']].head())


                  Block  total_counts
0     0000X W WACKER DR             0
1  023XX N LOCKWOOD AVE             0
2   055XX S ROCKWELL ST             0
3    045XX N ALBANY AVE             0
4   087XX S SANGAMON ST             0
